In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
import missingno as msno
# from pandas_profiling import ProfileReport
# https://stackoverflow.com/questions/64137932/create-interaction-term-in-scikit-learn

In [2]:
def OverSampler(parentDIR,df, xfilename, yfilename):
    # Oversample small class to balanced data
    os.chdir(parentDIR)
    df = df.dropna(how='all')
    # X = df.iloc[:,4:].copy()
    X = df.drop(['index','PatientID', 'TimeGroup','SerumID', 'Status', 'Rep'], axis=1).copy().reset_index(drop=True)
    # X['TimeGroup'] = df.TimeGroup.copy()
    X = pd.get_dummies(X)
    Y = df.Status.copy()

    ros = RandomOverSampler(random_state=0)
    Xoversampled, Yoversampled = ros.fit_resample(X, Y)

    # if not os.path.isdir('OverSampled'):
    #     os.makedirs('OverSampled')
    # os.chdir('OverSampled')

    Xoversampled.to_csv(xfilename, sep='\t')
    Yoversampled.to_csv(yfilename, sep='\t')
    return Xoversampled, Yoversampled

In [3]:
parentdir = os.getcwd()
testdata = pd.read_excel('Human Training and Testing sets for model.xlsx', sheet_name='Human Testing')
traindata = pd.read_excel('Human Training and Testing sets for model.xlsx', sheet_name='Human Training')
Full = pd.concat([testdata, traindata ]).reset_index()
Full.to_csv('Concatenated_RAW.txt', sep='\t')

In [6]:
Full.isna().sum()
Full.dropna(inplace=True)

### grouping Data
* finding average values from technical replicates. (Decided to use whole data and not average). 
* dropping whole cell lysate as it is not informative to antigens present for Burk infection.

In [7]:
# group = Full.groupby(['PatientID', 'SerumID', 'TimeGroup', 'Status', 'Type']).mean().reset_index()
Clean = Full.drop(['MSHR5855.WCL'], axis=1)
for name, group in Clean.groupby("Type"):
    print(name)
    print(Clean.Status.value_counts())

for name, group in group.groupby("Type"):
    print(name)
    print(Clean.TimeGroup.value_counts())

IgG
Negative    1436
Melioid     1428
Name: Status, dtype: int64
IgM
Negative    1436
Melioid     1428
Name: Status, dtype: int64
IgM
Healthy    1436
Week4+      963
Week2       180
Week3       163
Week1       106
Prior        16
Name: TimeGroup, dtype: int64


### NonResampled Data Splitting

In [ ]:
if not os.path.isdir('NonResampled'):
    os.makedirs('NonResampled')
os.chdir('NonResampled')
NonresDir = os.getcwd()

for name, group in Clean.groupby("Type"): # add SerumID to group
    print(name)
    os.chdir(NonresDir)
    if not os.path.isdir(name):
        os.makedirs(name)
    os.chdir(name)
    TypeDir = os.getcwd()
    if not os.path.isdir('ValidationSet'):
        os.makedirs('ValidationSet')
    os.chdir('ValidationSet')
    
    Week4 = group.loc[(group['TimeGroup']=='Week4+')].copy().sample(frac=0.1, random_state=0)
    Week3 = group.loc[(group['TimeGroup']=='Week3')].copy().sample(frac=0.1, random_state=0)
    Week2 = group.loc[(group['TimeGroup']=='Week2')].copy().sample(frac=0.1, random_state=0)
    Week1 = group.loc[(group['TimeGroup']=='Week1')].copy().sample(frac=0.1, random_state=0)
    prior = group.loc[(group['TimeGroup']=='Prior')].copy().sample(frac=0.1, random_state=0)
    Negative = group.loc[(group['TimeGroup']=='Healthy')].copy().sample(frac=0.1, random_state=0)

    ValidationSet = pd.concat([Week1, Week2, Week3, Week4, prior, Negative])
    ValidationSet.to_csv('ValidationSet.txt', sep='\t')
    print(ValidationSet.TimeGroup.value_counts())

    group.drop(Week1.index, inplace=True)
    group.drop(Week2.index, inplace=True)
    group.drop(Week3.index, inplace=True)
    group.drop(Week4.index, inplace=True)
    group.drop(Negative.index, inplace=True)
    group.drop(prior.index, inplace=True)


    os.chdir(TypeDir)
    Week4 = group.loc[(group['TimeGroup']=='Week4+')|(group['TimeGroup']=='Healthy')].copy()
    Week3 = group.loc[(group['TimeGroup']=='Week3')|(group['TimeGroup']=='Healthy')].copy()
    Week2 = group.loc[(group['TimeGroup']=='Week2')|(group['TimeGroup']=='Healthy')].copy()
    Week1 = group.loc[(group['TimeGroup']=='Week1')|(group['TimeGroup']=='Healthy')].copy()
    Prior = group.loc[(group['TimeGroup']=='Prior')|(group['TimeGroup']=='Healthy')].copy()
    Week4.to_csv('Week4_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    Week3.to_csv('Week3_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    Week2.to_csv('Week2_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    Week1.to_csv('Week1_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    Prior.to_csv('Prior_'+str(name)+'_PosNeg_grouped.txt', sep='\t')

In [ ]:
os.chdir(NonresDir)

igg4 = pd.read_csv('IgG/Week4_IgG_PosNeg_grouped.txt')
igg3 = pd.read_csv('IgG/Week3_IgG_PosNeg_grouped.txt')
igg2 = pd.read_csv('IgG/Week2_IgG_PosNeg_grouped.txt')
igg1 = pd.read_csv('IgG/Week1_IgG_PosNeg_grouped.txt')

igm4 = pd.read_csv('IgM/Week4_IgM_PosNeg_grouped.txt')
igm3 = pd.read_csv('IgM/Week3_IgM_PosNeg_grouped.txt')
igm2 = pd.read_csv('IgM/Week2_IgM_PosNeg_grouped.txt')
igm1 = pd.read_csv('IgM/Week1_IgM_PosNeg_grouped.txt')

if not os.path.isdir('IgGM'):
    os.makedirs('IgGM')
os.chdir('IgGM')
iggm1 = pd.concat([igg1,igm1])
iggm2 = pd.concat([igg2,igm2])
iggm3 = pd.concat([igg3,igm3])
iggm4 = pd.concat([igg4,igm4])

iggm1.to_csv("Week1_IgGM_PosNeg_grouped.txt", sep='\t')
iggm2.to_csv("Week2_IgGM_PosNeg_grouped.txt", sep='\t')
iggm3.to_csv("Week3_IgGM_PosNeg_grouped.txt", sep='\t')
iggm4.to_csv("Week4_IgGM_PosNeg_grouped.txt", sep='\t')

### OverSampled Data Cleaning and Splitting

In [8]:
os.chdir(parentdir)
if not os.path.isdir('OverSampled'):
    os.makedirs('OverSampled')
os.chdir('OverSampled')
OverDir = os.getcwd()

for name, group in Clean.groupby("Type"): # add SerumID to group
    print(name)
    os.chdir(OverDir)
    if not os.path.isdir(name):
        os.makedirs(name)
    os.chdir(name)
    TypeDir = os.getcwd()
    if not os.path.isdir('ValidationSet'):
        os.makedirs('ValidationSet')
    os.chdir('ValidationSet')

    Week4 = group.loc[(group['TimeGroup']=='Week4+')].copy().sample(frac=0.1, random_state=0)
    Week3 = group.loc[(group['TimeGroup']=='Week3')].copy().sample(frac=0.1, random_state=0)
    Week2 = group.loc[(group['TimeGroup']=='Week2')].copy().sample(frac=0.1, random_state=0)
    Week1 = group.loc[(group['TimeGroup']=='Week1')].copy().sample(frac=0.1, random_state=0)
    prior = group.loc[(group['TimeGroup']=='Prior')].copy().sample(frac=0.1, random_state=0)
    Negative = group.loc[(group['TimeGroup']=='Healthy')].copy().sample(frac=0.1, random_state=0)

    x1, y1 = OverSampler(os.getcwd(),pd.concat([Week1,Negative]), 'week1_'+str(name)+'predictor.txt', 'week1_'+str(name)+'response.txt')
    x2, y2 = OverSampler(os.getcwd(),pd.concat([Week2,Negative]), 'week2_'+str(name)+'predictor.txt', 'week2_'+str(name)+'response.txt')
    x3, y3 = OverSampler(os.getcwd(),pd.concat([Week3,Negative]), 'week3_'+str(name)+'predictor.txt', 'week3_'+str(name)+'response.txt')
    x4, y4 = OverSampler(os.getcwd(),pd.concat([Week4,Negative]), 'week4_'+str(name)+'predictor.txt', 'week4_'+str(name)+'response.txt')

    # ValidationSet = pd.concat([Week1, Week2, Week3, Week4, prior, Negative])
    # ValidationSet.to_csv('ValidationSet.txt', sep='\t')
    # print(ValidationSet.TimeGroup.value_counts())


    group.drop(Week1.index, inplace=True)
    group.drop(Week2.index, inplace=True)
    group.drop(Week3.index, inplace=True)
    group.drop(Week4.index, inplace=True)
    group.drop(Negative.index, inplace=True)
    group.drop(prior.index, inplace=True)


    os.chdir(TypeDir)
    Week4 = group.loc[(group['TimeGroup']=='Week4+')|(group['TimeGroup']=='Healthy')].copy()
    Week3 = group.loc[(group['TimeGroup']=='Week3')|(group['TimeGroup']=='Healthy')].copy()
    Week2 = group.loc[(group['TimeGroup']=='Week2')|(group['TimeGroup']=='Healthy')].copy()
    Week1 = group.loc[(group['TimeGroup']=='Week1')|(group['TimeGroup']=='Healthy')].copy()
    Prior = group.loc[(group['TimeGroup']=='Prior')|(group['TimeGroup']=='Healthy')].copy()

    print(Week4.Status.value_counts())
    
    X4, Y4 = OverSampler(TypeDir, Week4, 'W4_predictor.txt', 'W4_response.txt')
    X3, Y3 = OverSampler(TypeDir, Week3, 'W3_predictor.txt', 'W3_response.txt')
    X2, Y2 = OverSampler(TypeDir, Week2, 'W2_predictor.txt', 'W2_response.txt')
    X1, Y1 = OverSampler(TypeDir, Week1, 'W1_predictor.txt', 'W1_response.txt')


    # Week4.to_csv('Week4_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    # Week3.to_csv('Week3_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    # Week2.to_csv('Week2_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    # Week1.to_csv('Week1_'+str(name)+'_PosNeg_grouped.txt', sep='\t')
    # Prior.to_csv('Prior_'+str(name)+'_PosNeg_grouped.txt', sep='\t')

IgG
Negative    646
Melioid     435
Name: Status, dtype: int64
IgM
Negative    646
Melioid     432
Name: Status, dtype: int64


In [ ]:
os.chdir(OverDir)

igg4p = pd.read_csv('IgG/W4_predictor.txt')
igg3p = pd.read_csv('IgG/W3_predictor.txt')
igg2p = pd.read_csv('IgG/W2_predictor.txt')
igg1p = pd.read_csv('IgG/W1_predictor.txt')

igg4r = pd.read_csv('IgG/W4_response.txt')
igg3r = pd.read_csv('IgG/W3_response.txt')
igg2r = pd.read_csv('IgG/W2_response.txt')
igg1r = pd.read_csv('IgG/W1_response.txt')

igm4p = pd.read_csv('IgM/W4_predictor.txt')
igm3p = pd.read_csv('IgM/W3_predictor.txt')
igm2p = pd.read_csv('IgM/W2_predictor.txt')
igm1p = pd.read_csv('IgM/W1_predictor.txt')

igm4r = pd.read_csv('IgM/W4_response.txt')
igm3r = pd.read_csv('IgM/W3_response.txt')
igm2r = pd.read_csv('IgM/W2_response.txt')
igm1r = pd.read_csv('IgM/W1_response.txt')

if not os.path.isdir('IgGM'):
    os.makedirs('IgGM')
os.chdir('IgGM')
iggm1p = pd.concat([igg1p,igm1p])
iggm1r = pd.concat([igg1r,igm1r])
iggm2p = pd.concat([igg2p,igm2p])
iggm2r = pd.concat([igg2r,igm2r])
iggm3p = pd.concat([igg3p,igm3p])
iggm3r = pd.concat([igg3r,igm3r])
iggm4p = pd.concat([igg4p,igm4p])
iggm4r = pd.concat([igg4r,igm4r])

iggm1p.to_csv("W1_predictor.txt", sep='\t')
iggm1r.to_csv("W1_response.txt", sep='\t')
iggm2p.to_csv("W2_predictor.txt", sep='\t')
iggm2r.to_csv("W2_response.txt", sep='\t')
iggm3p.to_csv("W3_predictor.txt", sep='\t')
iggm3r.to_csv("W3_response.txt", sep='\t')
iggm4p.to_csv("W4_predictor.txt", sep='\t')
iggm4r.to_csv("W4_response.txt", sep='\t')

In [ ]:
Clean.describe()

In [ ]:
Clean.info()

In [ ]:
Clean.columns
Clean_min = ['BPSL1201_IMPS',
       'BPSL1404_ClpX', 'BPSL1743_Arg', 'BPSL2096_AhpC', 'BPSL2522_OmpA',
       'BPSL2697_GroEL', 'BPSL2827_DNAK', 'BPSL3222_rpIL', 'BPSL3396_AtpD',
       'BPSS0135', 'BPSS0476_GroS', 'BPSS0477_GroEL2', 'BPSS0530',
       'BPSS1498_HCP1.B', 'BPSS1652', 'BPSS1769_NADH', 'BPSS1850', 'CPS',
       'LPSA', 'LPSB']

In [ ]:
sns.set_style("whitegrid")
sns.set_palette('colorblind')
for col in Clean_min:
    fig,ax = plt.subplots()
    sns.boxplot(data=Clean, x=col, y='TimeGroup', ax=ax)
    plt.show()

In [ ]:
sns.pairplot(group, hue="Status", diag_kind="hist")

In [ ]:
sns.pairplot(group, hue="TimeGroup", diag_kind="hist")